# Problem

> We have a company with a delivery application and we seek to deliver orders in the shortest possible time, among all possible distributors to make the delivery. 

We have a defined logic to determine which distributor will be responsible for delivering each order, however in recent months we have noticed that delivery time has increased a lot, and we suspect that the problem lies in this logic and not in the distributors' operations.

# Solution

We can build an optimization model that minimizes delivery time. This problem is part of the class of transport problems.

As input we receive a matrix with the possible delivery times that each distributor would take to fulfill each order and as output we have the assignment of which orders each distributor must fulfill in a way that minimizes delivery time.

### Example

In [2]:
import pandas as pd
import numpy as np

In [3]:
# This time can be historical or a forecast from a separate delivery time model.
df_time_delivery = (
    pd.DataFrame([('distributor_1', 15, 10, 9, 2), 
                  ('distributor_2', 9, 15, 10, 3)], columns=['distributor', 'order_1', 'order_2', 'order_3', 'courier_available'])
      .set_index('distributor')
)

df_time_delivery

,order_1,order_2,order_3,courier_available
distributor,,,,
distributor_1,15,10,9,2
distributor_2,9,15,10,3


In [4]:
cost_matrix = df_time_delivery.drop(['courier_available'], axis=1).values
courier_available_matrix = np.array(df_time_delivery['courier_available'])

cost_matrix, courier_available_matrix

(array([[15, 10,  9],
        [ 9, 15, 10]]),
 array([2, 3]))

In this example the objective function would be:

$$\min Z = 15x_{11} + 10x_{12} + 9x_{13} + 9x_{21} + 15x_{22} + 10x_{23}$$

- We must ensure that an order will only be fulfilled by one distributor, because if more than one distributor needs to fulfill that order, we will incur additional shipping costs.

$$s.t.$$

$$x_{11} + x_{21} = 1$$
$$x_{12} + x_{22} = 1$$
$$x_{13} + x_{23} = 1$$

- **Case of orders grouped on the same route**: Minimization does not consider the factor of requests being fulfilled simultaneously or not. For example: if distributor 1 is responsible for delivering the 3 orders and only has 1 delivery person, the delivery time for the 2nd and 3rd orders will be distorted, as the delivery person will have taken a route that contains the time for the 2 other orders.
- We can follow the reasoning of always considering that orders will be fulfilled simultaneously in cases where we have more than 1 delivery person, and add this as a restriction. In our example we will consider that the 2 distributors have 3 delivery men available:

$$x_{11} + x_{12} + x_{13} <= 3$$
$$x_{21} + x_{22} + x_{23} <= 3$$

#### Generalization to any number of distributors and orders

In [5]:
def print_assignment(list_variables, n_distributor, n_orders):

    idx_selected = [i+1 for i, e in enumerate(list_variables) if e == 1]

    list_variables = []
    for distributor in range(1, n_distributor + 1):
        for order in range(1, n_orders + 1):
            list_variables.append(f'{distributor}{order}')
    
    from_to = dict(zip(range(1, n_distributor * n_orders + 1), list_variables))
    
    for idx in idx_selected:
        print(f'Order {from_to[idx][1]} assigned to Distributor {from_to[idx][0]}')

In [6]:
from pyomo.environ import *

def min_time_delivery_problem(cost_matrix: np.ndarray, solver_name : str = 'glpk', courier_available: np.ndarray = None):

    '''
    This function aims to minimize delivery time, given a set of distributors and orders to be served, 
    and thus determine which orders each distributor must fulfill simultaneously. 
    
    As a primary restriction we consider that an order can only be fulfilled by one distributor. 
    We also considered the option of adding restrictions related to the number of couriers available to fulfill the order simultaneously.
    
    :param cost_matrix: Time delivery matrix (distribution x order)
    :type cost_matrix: numpy.ndarray

    :param solver_name: Same pyomo SolverFactory, more information on possible values -> 
                   https://pyomo.readthedocs.io/en/stable/working_models.html
    :type solver_name: str

    :param courier_available (optional): Courier available (distribution x n_courier)
    :type courier_available: numpy.ndarray
    '''

    n_distributor, n_orders  = cost_matrix.shape[0], cost_matrix.shape[1]

    model = ConcreteModel()

    model.x = VarList(within=NonNegativeReals)
    for i in range(n_distributor*n_orders):
        model.x.add()

    model.obj = Objective(expr=sum([coef * model.x[idx] for coef, idx in zip(cost_matrix.reshape(-1), range(1, (n_distributor*n_orders) + 1))]), 
                          sense=minimize)

    # Restrictions that guarantee that an order will be fulfilled by only one distributor
    model.constraint_one_distributor_per_order = ConstraintList()
    for distributor in range(1, n_orders + 1):
        model.constraint_one_distributor_per_order.add(sum([model.x[idx] for idx in range(distributor, n_orders*n_distributor + 1, n_orders)]) == 1)

    if courier_available is not None:
        # Restrictions that ensure that the distributor only meets the maximum number of orders according to the 
        # maximum number of available couriers
        model.constraint_courier_available = ConstraintList()
        step = n_orders + 1
        distributor = 0
        for order in range(1, n_orders*n_distributor + 1, n_orders):
            model.constraint_courier_available.add(sum([model.x[idx] for idx in range(order, step)]) <= courier_available[distributor])
            step += n_orders
            distributor+=1

    solver = SolverFactory(solver_name)
    solver.solve(model)
    
    return model.obj(), [model.x[idx]() for idx in range(1, n_distributor*n_orders + 1)]

In [7]:
fun, x = min_time_delivery_problem(cost_matrix)

print(fun, x)

28.0 [0.0, 1.0, 1.0, 1.0, 0.0, 0.0]


In [8]:
print_assignment(x, cost_matrix.shape[0], cost_matrix.shape[1])

Order 2 assigned to Distributor 1
Order 3 assigned to Distributor 1
Order 1 assigned to Distributor 2


In [9]:
fun, x = min_time_delivery_problem(cost_matrix, courier_available = courier_available_matrix)

print(fun, x)

28.0 [0.0, 1.0, 1.0, 1.0, 0.0, 0.0]


In [10]:
print_assignment(x, cost_matrix.shape[0], cost_matrix.shape[1])

Order 2 assigned to Distributor 1
Order 3 assigned to Distributor 1
Order 1 assigned to Distributor 2


#### Specific to the example

In [11]:
model = ConcreteModel()

model.x11 = Var(within=NonNegativeReals)
model.x12 = Var(within=NonNegativeReals)
model.x13 = Var(within=NonNegativeReals)
model.x21 = Var(within=NonNegativeReals)
model.x22 = Var(within=NonNegativeReals)
model.x23 = Var(within=NonNegativeReals)

model.obj = Objective(expr=15*model.x11 + 10*model.x12 + 9*model.x13 + 9*model.x21 + 15*model.x22 + 10*model.x23, sense=minimize)

# Restrictions that guarantee that an order will be fulfilled by only one distributor
model.constraint1 = Constraint(expr=model.x11 + model.x21 == 1)
model.constraint2 = Constraint(expr=model.x12 + model.x22 == 1)
model.constraint3 = Constraint(expr=model.x13 + model.x23 == 1)

# Restrictions that ensure that the distributor only meets the maximum number of orders according to the maximum
# number of available couriers
model.courier_available1 = Constraint(expr=model.x11 + model.x12 + model.x13 <= 2) # 2 courier at distributor 1
model.courier_available2 = Constraint(expr=model.x21 + model.x22 + model.x23 <= 1) # 1 courier at distributor 1

solver = SolverFactory('glpk')
solver.solve(model)

print("Optimal Solution:")
print("Objective Value:", model.obj())
print("x11:", model.x11())
print("x12:", model.x12())
print("x13:", model.x13())
print("x21:", model.x21())
print("x22:", model.x22())
print("x23:", model.x23())

Optimal Solution:
Objective Value: 28.0
x11: 0.0
x12: 1.0
x13: 1.0
x21: 1.0
x22: 0.0
x23: 0.0


In [12]:
print_assignment([model.x11(), model.x12(), model.x13(), 
                  model.x21(), model.x22(), model.x23()], cost_matrix.shape[0], cost_matrix.shape[1])

Order 2 assigned to Distributor 1
Order 3 assigned to Distributor 1
Order 1 assigned to Distributor 2
